In [1]:
import glob
import os
from typing import Union
from datetime import datetime

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from omegaconf import OmegaConf

In [2]:
import copy

def deep_copy(func):
    def wrapper(*args, **kwargs):
        args_copy = copy.deepcopy(args)
        kwargs_copy = copy.deepcopy(kwargs)
        return func(*args_copy, **kwargs_copy)
    return wrapper

In [3]:
class DataObj:
    def __init__(self, data_path: Union[str, os.PathLike]):
        csv_paths = glob.glob(os.path.join(data_path, "**", "*.csv"))
        self.data = {os.path.basename(path).split('.')[0]: pd.read_csv(path, sep=';') for path in tqdm(csv_paths)}
        desc = pd.read_excel(glob.glob(os.path.join(data_path, "*.xlsx"))[0], sheet_name=None)
        desc.pop('Перечень файлов')
        self.desc = {key.split('.')[1]: desc[key] for key in desc}
        assert self.data.keys() == self.desc.keys(), "Keys must be equal"
        assert len(self.data.keys()) == 26, "Keys length must equals 26"
        self._keys = list(self.desc.keys())
        self._len = len(self._keys)

    def keys(self):
        return self._keys

    def __len__(self):
        return self._len

    def __getitem__(self, idx):
        return self._keys[idx]

In [4]:
config = {
    'seed':0xFACED,
    'device':'cpu',
    
    'paths':{
        'csv_path': './',
        
        'load_data': '', # /kaggle/input/base-data-with-features/base_data_with_features.csv
        
        'output_dir':'/content/',
    },
    
    'data':{
        'use_financial_features': True,
        'financial_codes': [1200, 1400, 1500, 1250, 1230, 2100, 2110, 2200, 2300, 2400, 1600, 1150],
        
        'use_complaint_features': True,
        'use_contract_features': True,
        'use_other_features': True,
        
        'cat_features': ['okved_basic_code_x', 'okved_basic_code_y']
    },
    
    'model':{
        
    }
}

config = OmegaConf.create(config)

In [5]:
full_data = DataObj(config.paths.csv_path)

  0%|          | 0/26 [00:00<?, ?it/s]

/tmp/ipykernel_56830/2116100949.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = {os.path.basename(path).split('.')[0]: pd.read_csv(path, sep=';') for path in tqdm(csv_paths)}
/tmp/ipykernel_56830/2116100949.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = {os.path.basename(path).split('.')[0]: pd.read_csv(path, sep=';') for path in tqdm(csv_paths)}
/tmp/ipykernel_56830/2116100949.py:4: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = {os.path.basename(path).split('.')[0]: pd.read_csv(path, sep=';') for path in tqdm(csv_paths)}


In [6]:
full_data.data["contract_main_info"] = full_data.data["contract_main_info"].dropna(subset=["supplier_inn"])

In [7]:
full_data.data["contract_main_info"]["supplier_inn"] = full_data.data["contract_main_info"]["supplier_inn"].astype(np.int64)

In [8]:
def str_value2int(x):
    x = str(x)
    if '-' in x:
        return 0
    elif x == 'nan':
        return 0
    elif x.startswith('(') and x.endswith(')'):
        return -int(x[1:-1])
    else:
        try:
            return int(x)
        except:
            return 0

In [9]:
def get_egrul_features(full_data):
    egrul_info = copy.deepcopy(full_data.data["egrul_info"])
    egrul_info["termination"] = ~egrul_info["termination_date"].isna()
    egrul_info["is_entity_person"] = egrul_info["entity_wo_attorney_type"] == "Физическое лицо"
    egrul_info["delta_days"] = (datetime.today() - pd.to_datetime(egrul_info["registration_date"])).dt.days
    egrul_info = egrul_info[["inn", "termination", "is_entity_person", "okved_basic_code", "delta_days", "has_filial", "capital_size"]]
    return egrul_info.set_index("inn")

In [10]:
def get_participation_features(full_data):
    df = copy.deepcopy(full_data.data["participation_statistic"])
    df = df.pivot_table(index='participant_inn',
                        columns='fz', values=['procedure_qty', 'win_qty'],
                        aggfunc='sum', fill_value=0)
    df.columns = ["_".join(col) for col in df.columns]
    df.index.names = ["inn"]
    return df

In [11]:
def get_bo_pivot(df, config):
    df = df.loc[df["str_code"].isin(config.data.financial_codes), ["inn", "str_code", "str_value"]]\
        .groupby(["inn", "str_code"]).agg(["sum"])["str_value"]
    df = df.reset_index()
    pivot_df = df.pivot_table(index='inn', columns='str_code', values='sum', aggfunc='sum', fill_value=0)
    pivot_df.columns = [f'str_code_{col}' for col in pivot_df.columns]
    return pivot_df

In [12]:
def get_financial_features(df):
    # Коэффициент текущей ликвидности
    df["current_liquid"] = df["str_code_1200"] / (df["str_code_1400"] + df["str_code_1500"])
    
    # Коэффициент быстрой ликвидности
    df["quick_liquid"] = (df["str_code_1250"] + df["str_code_1230"]) / (df["str_code_1400"] + df["str_code_1500"])
    
    # Валовая маржа
    df["gross_margin"] = df["str_code_2100"] / df["str_code_2110"]
    
    # Операционная маржа
    df["operating_margin"] = df["str_code_2200"] / df["str_code_2110"]
    
    # Маржа по прибыли до налогообложения
    df["profit_before_tax_margin"] = df["str_code_2300"] / df["str_code_2110"]
    
    # Чистая маржа
    df["net_margin"] = df["str_code_2400"] / df["str_code_2110"]
    
    # Коэффициент задолженности
    df["debt_ratio"] = (df["str_code_1400"] + df["str_code_1500"]) / df["str_code_1600"]
    
    # Фондоотдача
    df["asset_turnover"] = df["str_code_2110"] / df["str_code_1150"]
    
    return df

In [13]:
def get_contract_main_info_features(full_data):
    df = copy.deepcopy(full_data.data["contract_main_info"])\
                       .merge(copy.deepcopy(full_data.data["contract_termination"]), how="left", on="id_contract")
    df["is_termination"] = ~df["t_termination_date"].isna()
    
    info1 = df[["id_contract", "supplier_inn"]].groupby("supplier_inn").agg("count")
    info2 = df[["contract_price_rub", "supplier_inn", "is_termination"]].groupby("supplier_inn").agg("sum")
    
    info = pd.concat([info1, info2], axis=1)
    info.index.names = ["inn"]
    return info

In [14]:
def get_egrip_features(full_data):
    egrip_info = copy.deepcopy(full_data.data["egrip_info"])
    return egrip_info[["inn", "okved_basic_code"]].set_index("inn")

In [15]:
def get_complaint_features(full_data):
    df = copy.deepcopy(full_data.data["contract_main_info"])\
            .merge(copy.deepcopy(full_data.data["complaint_info"]),
                   how="inner", on="id_procedure")
    df = df[["supplier_inn", "status", "id_procedure"]].groupby(["supplier_inn", "status"]).agg(["count"])
    df.columns = ["_".join(col) for col in df.columns]
    df = df.reset_index()
    df = df.pivot_table(index='supplier_inn',
                        columns='status', values="id_procedure_count",
                        aggfunc='sum', fill_value=0)
    df.index.names = ["inn"]
    return df

In [16]:
def get_bo_features(full_data):
    bo_balance = copy.deepcopy(full_data.data["bo_balance"]) # бухгалтерский баланс
    bo_financial_results = copy.deepcopy(full_data.data["bo_financial_results"]) # отчет о финансовых результатах
    bo = pd.concat([bo_balance, bo_financial_results])
    bo["str_value"] = bo["str_value"].apply(str_value2int)
    bo["str_code"] = bo["str_code"].apply(lambda x: int(str(x).replace("*", "")))

    bo_pivot = get_bo_pivot(bo, config)
    bo_pivot = get_financial_features(bo_pivot)
    return bo_pivot

In [17]:
def get_avg_staff_qty_features(full_data):
    return full_data.data["avg_staff_qty"].set_index("inn")

In [18]:
bo_features = get_bo_features(full_data)

In [19]:
complaint_features = get_complaint_features(full_data)

In [20]:
egrul_features = get_egrul_features(full_data)

In [21]:
participation_features = get_participation_features(full_data)

In [22]:
contract_main_info_features = get_contract_main_info_features(full_data)

In [23]:
egrip_features = get_egrip_features(full_data)

In [24]:
complaint_features = get_complaint_features(full_data)

In [25]:
avg_staff_qty_features = get_avg_staff_qty_features(full_data)

In [26]:
bo_features.head(1)

,str_code_1150,str_code_1200,str_code_1230,str_code_1250,str_code_1400,str_code_1500,str_code_1600,str_code_2100,str_code_2110,str_code_2200,str_code_2300,str_code_2400,current_liquid,quick_liquid,gross_margin,operating_margin,profit_before_tax_margin,net_margin,debt_ratio,asset_turnover
inn,,,,,,,,,,,,,,,,,,,,
101003907,6499,9708,1247,3692,0,5640,16207,-6297,46568,-6297,-6506,-6506,1.721277,0.875709,-0.135222,-0.135222,-0.13971,-0.13971,0.347998,7.16541


In [27]:
complaint_features.head(1)

status,Отказано в рассмотрении,Отозвана,Рассмотрена
inn,,,
101009232,0,0,1


In [28]:
egrul_features.head(1)

,termination,is_entity_person,okved_basic_code,delta_days,has_filial,capital_size
inn,,,,,,
105041999,False,True,86.10,6961,True,NaN


In [29]:
participation_features.head(1)

,procedure_qty_223fz,procedure_qty_44fz,win_qty_223fz,win_qty_44fz
inn,,,,
101003907,1,14,1,11


In [30]:
contract_main_info_features.head(1)

,id_contract,contract_price_rub,is_termination
inn,,,
0,75,37003334.21,0


In [31]:
egrip_features.head(1)

,okved_basic_code
inn,
10100400827,45.32


In [32]:
complaint_features.head(1)

status,Отказано в рассмотрении,Отозвана,Рассмотрена
inn,,,
101009232,0,0,1


In [33]:
avg_staff_qty_features.head(1)

,avg_staff_qty
inn,
6164308645,3


In [ ]:
final_df = pd.concat([bo_features, complaint_features, egrul_features,
                      participation_features, egrip_features,
                      complaint_features, avg_staff_qty_features])

In [50]:
final_df = bo_features.join(complaint_features).join(egrul_features)
final_df = final_df.reset_index()
final_df["inn"].value_counts()

101003907     1
6829134550    1
6829136692    1
6829136396    1
6829136332    1
             ..
4345289849    1
4345288228    1
4345287305    1
4345286238    1
9909125356    1
Name: inn, Length: 96033, dtype: int64

In [43]:
final_df

,str_code_1150,str_code_1200,str_code_1230,str_code_1250,str_code_1400,str_code_1500,str_code_1600,str_code_2100,str_code_2110,str_code_2200,...,net_margin,debt_ratio,asset_turnover,Отказано в рассмотрении,Отозвана,Рассмотрена,procedure_qty_223fz,procedure_qty_44fz,win_qty_223fz,win_qty_44fz
inn,,,,,,,,,,,,,,,,,,,,,
101003907,6499.0,9708.0,1247.0,3692.0,0.0,5640.0,16207.0,-6297.0,46568.0,-6297.0,...,-0.139710,0.347998,7.165410,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101005862,1100.0,24.0,0.0,24.0,106.0,435.0,1124.0,-19.0,2777.0,-19.0,...,-0.006842,0.481317,2.524545,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101006993,1556.0,4585.0,4577.0,8.0,0.0,4920.0,6141.0,0.0,0.0,0.0,...,NaN,0.801172,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101007122,4870.0,8239.0,3707.0,969.0,0.0,12837.0,13301.0,-2455.0,15809.0,-2455.0,...,-0.155228,0.965115,3.246201,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101007130,0.0,14887.0,7740.0,23.0,0.0,46607.0,14887.0,-1627.0,6084.0,-2845.0,...,-0.450690,3.130718,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990103636632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,9.0,1.0,6.0
990116301104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,49.0,1.0,9.0
990116467452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,57.0,0.0,11.0


In [ ]:
final_df[final_df["inn"].isin(full_data.data["inn_list"]["inn"])]